In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import re
import requests
import time
from selenium.common.exceptions import StaleElementReferenceException

In [92]:
def getdriver(): #it is the selenium driver to connect the script with the browser
    options = webdriver.ChromeOptions()
    options.add_argument('log-level=3')
    options.add_argument("--window-size=1880x1020")
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-gpu')
    #options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    options.add_experimental_option("prefs",prefs)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("test-type")
    options.add_argument("--disable-web-security")
    options.add_argument("--allow-running-insecure-content")
    driver=webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())
    #     driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """
    Object.defineProperty(navigator, 'webdriver', {
        get: () => undefined
    })
    """})
    return driver
driver = getdriver()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Pfactorial00110\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_2712\4128986626.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())


In [13]:
job_link = []
Category = []
def extract_job_post(): # this function extracts the particular data of the mentioned website
   try: 
      for i in range(0,10):
         url_next="https://jobs.freshershome.com/page"+str(i)+"?action=search&5=machine+learning" #url to extract
         driver.get(url_next) #connects to the browser
         content = driver.page_source # stores page source in a variable
         soups=BeautifulSoup(content, 'html.parser') 
         soup=soups.findAll('a',href=True) # finds all the a tags in the respective html
         for item in soup:
            if item['href'].startswith('https://jobs.freshershome.com/detail/'): #to extract the particular data which starts with particular words
               job_link.append(item['href']) #to add the extracted data to the empty list
               Category.append('Machine Learning')
   except:
      pass
   return job_link,Category

In [14]:
job_link, Category = extract_job_post()

In [16]:
Job_posting_df=pd.DataFrame({"Category":Category,"Job_Posting_link":job_link})

In [12]:
Job_posting_df.to_csv('Data_science_freshershome_links.csv', index=False)

In [17]:
Job_posting_df.to_csv('Machine_learning_freshershome_links.csv', index=False)

In [93]:
DS = pd.read_csv('Data_science_freshershome_links.csv')
ML = pd.read_csv('Machine_learning_freshershome_links.csv')

In [112]:
def fresh_details_extractor(df): # this function extracts the data from the particular job link
    carbs_df = pd.DataFrame()
    for i in range(0,len(df)): #it iterates through the links upto the extracted links length
        try:
          Job_Posting_link=df['Job_Posting_link'][i] #iterates through the links in the dataframe
          Category=df['Category'][i]
          driver.get(Job_Posting_link)
          #content = driver.page_source
          #soups1=BeautifulSoup(content,'html.parser')
          details_list = [] 
          details = driver.find_element(by=By.CLASS_NAME, value='detail-list').find_elements(by=By.TAG_NAME, value='span')
          for i in details:
            details_list.append(i.text)
          try:
            Location= details_list[1] #driver.find_element(by=By.XPATH, value='/html/body/div/section/div/div[1]/div[1]/div[5]/div[2]/ul/li[2]/span').text #extracts the text data from the respective class                                   #/html/body/div[1]/div[2]/main/div[1]/section[1]/div[3]/div[1]/div[1]/div[1]/p[2]
          except:
            Location=None                                      #/html/body/div/section/div/div[1]/div[1]/div[7]/div[1]/h4
          try:
            company = driver.find_element(by=By.XPATH, value="/html/body/div/section/div/div[1]/div[1]/div[6]/div[1]/h4").text #soups1.find(class_ = 'info-org').text
          except:
            company=None
          try:
            title = details_list[3] #driver.find_element(by=By.XPATH, value="/html/body/div/section/div/div[1]/div[1]/div[5]/div[2]/ul/li[4]/span").text #soups1.find(class_ = 'info-position').text
          except:
            title=None
          try:
              salary = details_list[4] #salary = driver.find_element(by=By.XPATH, value='/html/body/div/section/div/div[1]/div[1]/div[5]/div[2]/ul/li[5]/span').text
          except:
            salary=None
          try:                                                    
            description = driver.find_element(by=By.XPATH, value='//*[@id="job-details"]/div[2]').text #soups1.find(class_='job-description').text 
          except:
            description=None
          try:
            experience = details_list[0] #driver.find_element(by=By.XPATH, value='/html/body/div/section/div/div[1]/div[1]/div[5]/div[2]/ul/li[1]/span').text
          except:
            experience=None
          try:
            education = details_list[2] #driver.find_element(by=By.XPATH, value='/html/body/div/section/div/div[1]/div[1]/div[5]/div[2]/ul/li[3]/span').text #('//div[@class="col"][3]').text #/html/body/div[1]/div[2]/main/div[1]/section[1]/div[3]/div[1]/div[1]/div[3]/p[2]
          except:
            education=None
          try:
            skills = details_list[6]
          except:
            skills = None
          carbs=pd.DataFrame({'Category':Category,'Job_Posting_link':Job_Posting_link,'Job_title':title,'Company_name':company,'Location':Location,'Education':education,'Salary':salary,'Experience':experience,'Skills':skills,'Description':[description]},index=[0])
          carbs_df=carbs_df.append(carbs)
        except Exception as e: # raises an exception if something goes wrong in the time of extraction
          print(e)
    return carbs_df 

In [ ]:
Data_science = fresh_details_extractor(DS)

In [ ]:
Machine_learning = fresh_details_extractor(ML)

In [118]:
Data_science.to_csv('fresherhome_DS_full_details.csv', index=False)

In [115]:
Machine_learning.to_csv('fresherhome_ML_full_details.csv', index=False)